# Analysis Summary

Our goal from this analysis will be to predict the outcome of March Madness games. To do this, we will be using a random forest classifer that takes in historical data from NCAA tourney matchups as input and outputs predictions for the winning team for each game in an input set of March Madness games. 

### Baseline Model
The baseline model we will use to compare our model's results against will be solely tied to a team's RPI. Our baseline model poses the following hypothesis:

    That in any NCAA tournament game, the team with the lower RPI will win the game.
  
Intuitively, this is a reasonable prediction. RPI (Ratings Percentage Index), ranks teams based on their wins, losses, and strength of schedule for the past season. If team A has a lower RPI than team B at the end of a season, it's generally considered that team A's performance throughout the season has been at a higher level than team B's. For this reason, if we knew nothing else about the two teams, predicting the outcome based on RPI is a good starting point. 


### Data Sources Overview
The data the was used to perform this analysis came from the Google Cloud & NCAA® ML Competition 2018-Men's Kaggle competition (Insert Link). The following is a brief summary of all the datasets that were used:
- NCAATourneyCompactResults: Contains records from each NCAA tournament game from 1985-2017, including score and region.  
- Teams: Contains information for each Division 1 (D1) basketball team including an ID, name, first and most recent year playing D1 basketball. 
- MasseyOrdinals_Prelim2018: Contains data from 2003-2017 surrounding each D1 team's rank from various rankings sources throughout. 
- RegularSeasonDetailedResults: Contains similar information as the NCAATourneyCompactResults dataset, with the addition that each row will also contain the totals in a variety of statistical categories for the winning team and losing team. These are categories that are often found in a boxscore. 

A more in depth description of each of the datasets that were used and additional datasets provided by Kaggle can be found at https://www.kaggle.com/c/mens-machine-learning-competition-2018/data


### Data Cleansing & Preparation

In order to create our random forest, we need to complete the following:
    1. Settle on a set of statistics to evaluate the performance of each team. 
    2. Create a data frame that represents a matchup that occurred in an NCAA tournament and contains our set of statistics for each team in the matchup 

To accomplish the first task, the following statistics were calculated for each team:
    - Defensive Rebounds (DRB)
    - Efficient Field Goal Percentage (EFG)
    - Free Throws Attempted (FTA)
    - Free Throw Percentage (FTP)
    - Margin of Loss (MOL)
    - Margin of Victory (MOV)
    - Offensive Rebounds (ORB)
    - Possessions (POSS)
    - Turnovers (TO)
    - Turnovers Forced (TOF)
    - Defensive Efficiency (dEff)
    - Offensive Effiency (oEff)
    - Rankings Percentage Index (RPI)
    - Conference Tourney Wins (confTournWins)
    - Wins vs. a Tournament Team (winsVsTourney)
    - Number of Games Played (numGamesPlayed)


With the exception of the last four, the yearly pre-tournament averages (regular season and conference tourney play) were found. The last three are cumulative totals, and RPI was found by using the latest ranking given to a team before the tournament. 

By parsing through the 4 datasets listed above, we can calculate the desired information for each team in a March Madness tournament from 2003-2017 as seen below:

In [74]:
import pandas as pd 
from scipy import stats as sciStats
import numpy as np
import team, game as g
import Model as mod

In [75]:
def getTeamNames():
    """
    Return dictionary where key is team ID and value is team name
    """
    names = {}
    teams = pd.read_csv("Data/Teams.csv")
    for index, row in teams.iterrows():
        teamId = row["TeamID"]
        name = row["TeamName"]
        names[teamId] = name
    return names

def getSeasonStats(ncaaTourneyTeams):
    """
    Use regular season results and RPI rankings to create a 
    dictionary where key is the team's ID and the value is a 
    Team object. Team objects contain yearly avg stats for each 
    team in various categories.
    """
    teams = {}
    names = getTeamNames()
    unfiltRanks = pd.read_csv("data/MasseyOrdinals_Prelim2018.csv")
    rankings = unfiltRanks[(unfiltRanks["SystemName"] == "RPI") & (unfiltRanks["RankingDayNum"] == 133)]
    regSeasonResults = pd.read_csv("data/RegularSeasonDetailedResults.csv")
    for index, row in regSeasonResults.iterrows():
        season = row["Season"]
        dayNum = row["DayNum"]
        wTeamId = row["WTeamID"]
        lTeamId = row["LTeamID"]
        customWId = str(wTeamId) + "_" + str(season)
        customLId = str(lTeamId) + "_" + str(season)
        wRPI = None
        lRPI = None
        try:
            wRPI = rankings[(rankings["Season"] == season) & (rankings["TeamID"] == wTeamId)].iloc[0]["OrdinalRank"]
            lRPI = rankings[(rankings["Season"] == season) & (rankings["TeamID"] == lTeamId)].iloc[0]["OrdinalRank"]
        except Exception as e:
            pass
            # print str(lTeamId) + " " + str(season) + " not found"
        
        if customWId not in teams:
            teams[customWId] = team.Team(customWId)
        if customLId not in teams:
            teams[customLId] = team.Team(customLId)
        wTeam = teams[customWId]
        wTeam.RPI = wRPI
        wTeam.name = names[wTeamId]
        wTeam.updateStats(row, True)
        if customLId in ncaaTourneyTeams:
            wTeam.winsVsTourney += 1
        lTeam = teams[customLId]
        lTeam.name = names[lTeamId]
        lTeam.RPI = lRPI
        lTeam.updateStats(row, False)
        
    for t in teams:
        teams[t].setScore()
    
    return teams

def getTeamStats(teams):
    """
    Get season stats for each team in a Data Frame. Able to see yearly averages and totals for stored
    statistical categories in each team object in teams dictionary
    """
    allTeamData = []
    visited = set()
    for team in teams:
        if team not in visited:
            idAndSeason = team.split("_")
            season = idAndSeason[1]
            _id = idAndSeason[0]
            teamData = teams[team].objToDict()
            teamData['season'] = season
            allTeamData.append(teamData)
            visited.add(team)
    return pd.DataFrame(allTeamData)

def dataToDict(df):
    """
    Convert dataframe to list to dictionary where key is team id and value is row data
    """
    data = {}
    for index, row in df.iterrows():
        data[row["_id"]] = row.to_dict()
    return data

def normalizeFeatures(teamStats):
    """
    Uses team stats for all seasons and returns data frame with stats normalized by season
    """
    toReturn = teamStats.copy()
    copy = toReturn.copy()
    copy['season'] = copy['season'].astype(int)
    copy.drop(copy.select_dtypes(['object']), inplace=True, axis=1)
    normalized = normalize(copy, 'season')
#     normalized.drop(columns=['numGamesPlayed'])
    toReturn.update(normalized)
    return toReturn

def normalize(df, by):
    """
    groups df by season and normalizes each statistical category of features from 0 - 1
    """
    groups = df.groupby(by)
    mins = groups.transform(np.min)
    maxs = groups.transform(np.max)
    return (df[mins.columns] - mins) / (maxs - mins)

def populateNCAATourneyTeams():
    """
    Create an ID for each team using a combination of its id and the season the team played in. 
    Output a dictionary with an entry for each team whose key is its newly created id
    """
    ncaaTourneyTeams = {}
    ncaaTournResults = pd.read_csv("data/NCAATourneyCompactResults.csv")
    for index, row in ncaaTournResults.iterrows():
        season = row["Season"]
        dayNum = row["DayNum"]
        wTeamId = row["WTeamID"]
        lTeamId = row["LTeamID"]
        customWId = str(wTeamId) + "_" + str(season)
        customLId = str(lTeamId) + "_" + str(season)

        if customWId not in ncaaTourneyTeams:
            ncaaTourneyTeams[customWId] = 1
        if customLId not in ncaaTourneyTeams:
            ncaaTourneyTeams[customLId] = 1
    return ncaaTourneyTeams

def getMatchups(teams, inputDataModified=False):
    """
    Use NCAA Tournament results to return data frame of matchups where each row contains data for one matchup between two teams, including their yearly avg totals in statistical categories, RPI, and game result.
    """
    matchups = []
    ncaaTournResults = pd.read_csv("data/NCAATourneyCompactResults.csv")
    for index, row in ncaaTournResults.iterrows():
        season = row["Season"]
        dayNum = row["DayNum"]
        wTeamId = row["WTeamID"]
        lTeamId = row["LTeamID"]
        customWId = str(wTeamId) + "_" + str(season)
        customLId = str(lTeamId) + "_" + str(season)

        if customWId in teams and customLId in teams:
            wTeamData, lTeamData = {}, {}
            if inputDataModified:
                wTeamData = teams[customWId].copy()
                del wTeamData['season']
            else:
                wTeamData = teams[customWId].objToDict().copy()
            wTeamDataMod, lTeamDataMod = {}, {}
            for key in wTeamData.keys():
                wTeamDataMod["w" + key] = wTeamData[key]
                
            if inputDataModified:
                lTeamData = teams[customLId]
                del lTeamData['season']
            else:
                lTeamData = teams[customLId].objToDict().copy()
            for key in lTeamData.keys():
                lTeamDataMod["l" + key] = lTeamData[key]
                
            matchupData = wTeamDataMod.copy()
            matchupData.update(lTeamDataMod)
            matchupData["dayNum"] = dayNum
            matchupData["season"] = season
            matchups.append(matchupData)
    colOrder = ["dayNum", "season", "l_id", "lname", "w_id", "wname", "lscore", "wscore", "lDRB", "lEFG", "lFTA", "lFTP", "lMOL", "lMOV", "lORB", "lPOSS",
                "lRPI", "lTO", "lTOF", "lconfTournWins", "ldEff", "lnumGamesPlayed", "loEff", "lwinsVsTourney",
                "wDRB", "wEFG", "wFTA", "wFTP", "wMOL", "wMOV", "wORB", "wPOSS", "wRPI", "wTO", "wTOF", 
                "wconfTournWins", "wdEff", "wnumGamesPlayed", "woEff", "wwinsVsTourney"]
    df = pd.DataFrame.from_dict(matchups)
    df = df[colOrder]
    return df

def getMatchupData():
    """
    Returns data frame of historical matchups in NCAA tournament.
    Reads in existing CSV if available. Otherwise, produces data frame by creating Team objects, calculating yearly avg totals for each team, and joining with historical NCAA tourney matchup data
    """
    try:
        matchups = pd.read_csv("Data/output/matchups_normalized.csv")
        return matchups
    except Exception as e:
        ncaaTourneyTeams = populateNCAATourneyTeams()
        teamObjs = getSeasonStats(ncaaTourneyTeams)
        teamStats = getTeamStats(teamObjs)
        normalized = normalizeFeatures(teamStats)
        dataAsDict = dataToDict(normalized)
        matchups = getMatchups(dataAsDict, inputDataModified=True)
        matchups.to_csv("Data/output/matchups_normalized.csv", index=False)
        return matchups

In [76]:
def getRankings():
    unfiltRanks = pd.read_csv("data/MasseyOrdinals_Prelim2018.csv")
    rankings = unfiltRanks[(unfiltRanks["SystemName"] == "POM") & (unfiltRanks["RankingDayNum"] == 133) | (unfiltRanks["SystemName"] == "RPI") & (unfiltRanks["RankingDayNum"] == 133)]
    transpose = {}
    for index, row in rankings.iterrows():
        teamId = str(row["TeamID"]) + "_" + str(row["Season"])
        if teamId in transpose:
            if row["SystemName"] == "RPI":
                transpose[teamId]["RPI"] = row["OrdinalRank"]
            else:
                transpose[teamId]["POM"] = row["OrdinalRank"]
        else:
            transpose[teamId] = {}
            if row["SystemName"] == "RPI":
                transpose[teamId]["RPI"] = row["OrdinalRank"]
            else:
                transpose[teamId]["POM"] = row["OrdinalRank"]
    ranksDF = pd.DataFrame.from_dict(orient = "index", data = transpose).reset_index()
    ranksDF.columns = ["teamId", "POM", "RPI"]
    return ranksDF

def getSeeds():
    seeds = pd.read_csv("data/NCAATourneySeeds.csv")
    seeds["teamId"] = seeds.TeamID.astype(str).str.cat(seeds.Season.astype(str), sep='_')
    seeds["Seed"] = seeds.Seed.str.extract('(\d+)', expand=False).astype(int)
    seeds = seeds.drop(["Season", "TeamID"], axis = 1)
    return seeds

def mergeSeedsAndRanks(seeds, ranks):
    return seeds.merge(ranks, how = "inner", on = "teamId")

def mergeSeedsRanksStats(seedsAndRanks, stats):
    return stats.merge(seedsAndRanks, how = "inner", on = "teamId")

def normalizeFeatures(teamStats):
    """
    Uses team stats for all seasons and returns data frame with stats normalized by season
    """
    toReturn = teamStats.copy()
    copy = toReturn.copy()
    copy['season'] = copy['season'].astype(int)
    copy.drop(copy.select_dtypes(['object']), inplace=True, axis=1)
    normalized = normalize(copy, 'season')
    toReturn.update(normalized)
    return toReturn

def normalize(df, by):
    """
    groups df by season and normalizes each statistical category of features from 0 - 1
    """
    groups = df.groupby(by)
    mins = groups.transform(np.min)
    maxs = groups.transform(np.max)
    return (df[mins.columns] - mins) / (maxs - mins)

def findScore(teamA, teamB):
    offScoreData = [5*teamA["oEff"].values[0],(1 - teamA["POS"] - teamB["POS"].values[0]).values[0],2*teamA["3PP"].values[0],3*teamA["3P%"].values[0],8*teamA["FG%"].values[0],5*teamA["FTA"].values[0],3.5*teamA["FT%"].values[0],2*teamA["OR"].values[0],2.5*teamA["TO"].values[0]]
    defScoreData = [8.5*(1-teamA["dEff"]).values[0], 5*teamA["DR"].values[0], 3*teamA["TOF"].values[0], 5*teamA["PF"].values[0]]
    seedData = [12.5*(1-teamA["Seed"]).values[0], 10*(1-teamA["POM"]).values[0], 7.5*(1-teamA["RPI"]).values[0]]
    offScore = (5*teamA["oEff"]*(1 - teamA["POS"] - teamB["POS"].values[0]) + 4*teamA["3P%"] + 8*teamA["2P%"] + 5*teamA["FTA"] + 3.5*teamA["FT%"] + 2*teamA["OR"] - 2.5*teamA["TO"])
    defScore = (8.5*(1-teamA["dEff"]) + 5*teamA["DR"] + 3*teamA["TOF"] - 5*teamA["PF"])
    seedScore = 12.5*(1-teamA["Seed"]) + 10*(1-teamA["POM"]) + 7.5*(1-teamA["RPI"])
    total = offScore.values[0] + defScore.values[0] + seedScore.values[0]
    scoreData = [teamA["teamId"].values[0]] + offScoreData + [offScore.values[0]] + defScoreData + [defScore.values[0]] + seedData + [seedScore.values[0]] + [total]
    return scoreData


In [77]:
def setupStatsDF(year):
    allGames = []
    regSeasonResults = pd.read_csv("data/RegularSeasonDetailedResults.csv")
    games = regSeasonResults[regSeasonResults["Season"] == year]
    for index, row in games.iterrows():
        season = row["Season"]
        dayNum = row["DayNum"]
        wTeamId = row["WTeamID"]
        lTeamId = row["LTeamID"]
        customWId = str(wTeamId) + "_" + str(season)
        customLId = str(lTeamId) + "_" + str(season)
        row1 = [customWId, season, dayNum, row["WScore"], row["LScore"], row[30]] + list(row[8:21])
        row2 = [customLId, season, dayNum, row["LScore"], row["WScore"], row[17]] + list(row[21:])
        allGames.append(row1)
        allGames.append(row2)
    colNames = ["teamId", "season", "dayNum", "score", "oppScore", "TOF", "FGM", "FGA", "FGM3", "FGA3", "FTM", "FTA", "OR", "DR", "Ast", 
               "TO", "Stl", "Blk", "PF"]
    gamesDF = pd.DataFrame(allGames, columns = colNames)
    gamesDF = gamesDF.drop(["dayNum"], axis = 1)
    gamesDF["season"] = gamesDF["season"].astype("object")
    gamesDF["teamId"] = gamesDF["teamId"].astype("object")
    statDevs = gamesDF.groupby(by = "teamId").std().reset_index() ## Consider using later
    gamesDF["POS"] = gamesDF["FGA"] - gamesDF["OR"] + gamesDF["TO"] + (0.4 * gamesDF["FTA"])
    sums = gamesDF.groupby(by = "teamId").sum().reset_index()
    means = gamesDF.groupby(by = "teamId").mean().reset_index()

    sums["oEff"] = sums["score"]/sums["POS"]
    sums["dEff"] = sums["oppScore"]/sums["POS"]
    sums["eFG"] = (sums["FGM"] + (0.5 + sums["FGM3"])) / sums["FGA"]
    sums["FG%"] = sums["FGM"]/sums["FGA"]
    sums["3P%"] = sums["FGM3"]/sums["FGA3"]
    sums["3PP"] = 3*sums["FGM3"]/sums["score"]
    sums["FT%"] = sums["FTM"]/sums["FTA"]
    sums["2P%"] = (sums["FGM"] - sums["FGM3"]) / (sums["FGA"] - sums["FGA3"])
    sums["2PP"] = 1 - sums["3PP"]

    meanSub = means[["teamId", "FTA", "OR", "DR", "TO", "TOF", "POS", "PF"]]
    sumSub = sums[["teamId", "oEff", "dEff", "eFG", "FG%", "3P%", "FT%", "3PP", "2P%", "2PP"]]
    statsSub = sumSub.merge(meanSub, how='left', left_on = "teamId", right_on = "teamId")
    statsSub["season"] = stats["teamId"].apply(lambda x: x[x.index("_") + 1:])
    return statsSub

In [ ]:
ranks = getRankings()
seeds = getSeeds()
seedsAndRanks = mergeSeedsAndRanks(seeds, ranks)
names = getTeamNames()
matchups = []
teamScoreInfo = []
visited = set()
for year in range(2003, 2018):
    statsDF = setupStatsDF(year)
    statsSeeds = mergeSeedsRanksStats(seedsAndRanks, statsDF)
    normStats = normalizeFeatures(statsSeeds)
    tournRes = pd.read_csv("data/NCAATourneyCompactResults.csv")
    tournRes = tournRes[tournRes["DayNum"] > 135]
    recentRes = tournRes[tournRes["Season"] == year]

    for index, row in recentRes.iterrows():
        season = row["Season"]
        wTeamId = row["WTeamID"]
        lTeamId = row["LTeamID"]
        customWId = str(wTeamId) + "_" + str(season)
        customLId = str(lTeamId) + "_" + str(season)
        teamW = normStats[normStats["teamId"] == customWId]
        teamL = normStats[normStats["teamId"] == customLId]
        seedRankW = [teamW["RPI"].values[0], teamW["POM"].values[0], teamW["Seed"].values[0]]
        seedRankL = [teamL["RPI"].values[0], teamL["POM"].values[0], teamL["Seed"].values[0]]
        if not teamW.empty:
            wScore = findScore(teamW, teamL)
            lScore = findScore(teamL, teamW)
            if customWId not in visited:
                teamScoreInfo.append(wScore)
                visited.add(customWId)
            if customLId not in visited:
                teamScoreInfo.append(lScore)
                visited.add(customLId)
            matchups.append([season, customWId, names[wTeamId], customLId, names[lTeamId], wScore[-1], lScore[-1]] + seedRankW + seedRankL)
matchDF = pd.DataFrame(matchups, columns = ["season", "wId", "wName", "lId", "lName", "wScore", "lScore", 
                                           "wRPI", "wPOM", "wSeed", "lRPI", "lPOM", "lSeed"])
matchDF["predicted"] = matchDF["wScore"] > matchDF["lScore"]
matchDF["seedBaseline"] = matchDF["wSeed"] < matchDF["lSeed"]
matchDF["pomBaseline"] = matchDF["wPOM"] < matchDF["lPOM"]
matchDF["rpiBaseline"] = matchDF["wRPI"] < matchDF["lRPI"]
matchDF.to_csv("data/output/matchupRes_regression.csv", index = False)
                                  

In [68]:
# no less than for seedbaseline
modelComparisons = matchDF.groupby('season')["predicted", "seedBaseline", "pomBaseline", "rpiBaseline"].agg('sum')

In [69]:
myModel = np.array(modelComparisons ["predicted"])
seed = np.array(modelComparisons["seedBaseline"])
np.random.seed(12345678)
sciStats.ttest_rel(myModel,seed)

Ttest_relResult(statistic=2.1754067128234165, pvalue=0.047224674617508226)